```
Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Setup

Welcome to the Keywords and Ads Translator Colab. This Colab uses Google Ads API to connect to your Google Ads accounts and allows you to translate all keywords in selected accounts and campaigns from one language to another and modifies campaign and adgroup names according to the chosen target language. The output is formatted to Google Ads Editor templates which you can upload directly to Google Ads.

You need the following to get started:

1. A Google Cloud Project with billing enabled and oAuth Consent Screen configured (see details [here](https://developers.google.com/workspace/guides/configure-oauth-consent))
2. A Google Ads Developer Token from your Google Ads MCC API Center.
3. A Login Customer ID (typically the top level MCC).

Once you have those, you can run the below cells in consecutive order and following the on screen instructions.

In [ ]:
#@title Install required libraries
#@markdown After running this cell the **runtime will crash**, this is expected
#@markdown behavior and you can proceed to Load Libraries.
!pip install progress
import os
import time
from progress.bar import Bar
from IPython.display import HTML, display

packages = [
    'alive_progress',
    'google-ads==21.0.0',
    'google-api-python-client',
    'google-auth-httplib2',
    'google-auth-oauthlib',
    'google-cloud-translate',
    'google-cloud-storage',
    'gspread==5.2.0']

max = len(packages)

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 40%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

counter = 0
out = display(progress(0, max), display_id=True)

for package in packages:
    !pip install -q {package}
    print(f'Installing {package}')
    out.update(progress(counter, max-1))
    counter += 1
print('Killing runtime. Proceed to next cell.')
time.sleep(1)
os.kill(os.getpid(), 9)

In [ ]:
#@title Load libraries
import pandas as pd
import ipywidgets as widgets
import math
import logging
import enum
import time
import itertools
import re
import os

import google.oauth2.credentials
import google_auth_oauthlib.flow
from google.colab import files
from oauthlib.oauth2.rfc6749.errors import InvalidGrantError
from urllib import parse
from typing import Optional, Union, Any
from concurrent import futures
from IPython.display import display
from google.colab import auth
from google.oauth2.credentials import Credentials
from google.auth import default
import gspread
from alive_progress import alive_it
from google.cloud import translate_v3

from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

auth.authenticate_user()

In [ ]:
#@title Set Cloud Project and enable APIs
#@markdown Enter the Google Cloud Project ID you want to use in order to enable the required APIs and follow on screen instructions.

cloud_project_id = '' #@param {type:"string"}

!gcloud auth application-default login
print(f'Updating project to {cloud_project_id}...')
!gcloud config set project $cloud_project_id

print('Enabling required APIs...')
!gcloud services enable translate.googleapis.com
!gcloud services enable sheets.googleapis.com
!gcloud services enable drive.googleapis.com
!gcloud services enable googleads.googleapis.com
print('Success')

In [ ]:
#@title Authentication
#@markdown Please follow the below steps to setup authentication.
#@markdown 1.   Go to a [Google Cloud Project](https://console.cloud.google.com/) and navigate to `APIs & Services > Credentials > Create Credentials` to generate an oAuth client ID.
#@markdown Select `Web Application` and add `http://localhost:8080` to the "Authorized redirect URIs" and hit "Create".
#@markdown 2.    Download the `client_secrets.json` file.
#@markdown 3.    Set the `Google Ads developer token` and `Login Customer ID` in the fields below.
#@markdown 4.    Run the below section and follow the instruction prompts. Hit `Check Authentication`button below after to ensure authentication is working.
uploaded = files.upload()
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/cloud-translation',
          'https://www.googleapis.com/auth/adwords']

filename = [file for file in uploaded][0]
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    filename,
    scopes=SCOPES)

flow.redirect_uri = 'http://localhost:8080'
authorization_url, state = flow.authorization_url(
    # Enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true',
    prompt='consent')
print('\n-----------------------------------------------------------')
print('Log into the Google Account you use to access your AdWords account '
      'and go to the following URL: \n%s\n' % authorization_url)
print('-----------------------------------------------------------')
print('After approving you will ecounter ERR_CONNECTION_REFUSED - This is expected.')
print('Copy and paste the full URL from your browsers address bar.')
url = input('URL: ').strip()
code = parse.parse_qs(parse.urlparse(url).query)['code'][0]
os.environ['OAUTHLIB_RELAX_TOKEN_SCOPE'] = '1'
try:
    flow.fetch_token(code=code)
except InvalidGrantError as ex:
    print('Authentication has failed: %s' % ex)
print('Access token: %s' % flow.credentials.token)
print('Refresh token: %s' % flow.credentials.refresh_token)

developer_token= '' #@param {type:"string"}
login_customer_id= '' #@param {type:"string"}

config = {
    "client_id": flow.credentials.client_id,
    "client_secret": flow.credentials.client_secret,
    "developer_token": developer_token.strip(),
    "login_customer_id": login_customer_id.strip(),
    "refresh_token": flow.credentials.refresh_token,
    "use_proto_plus": True
}

button = widgets.Button(description="Check Authentication")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    if config['client_id'] and config['client_secret'] and config['refresh_token'] and config['developer_token'] and config['login_customer_id']:
      try:
        client = GoogleAdsClient.load_from_dict(config)
        ga_service = client.get_service("GoogleAdsService")
        query = "SELECT customer.id FROM customer LIMIT 1"
        request = client.get_type("SearchGoogleAdsRequest")
        request.customer_id = login_customer_id
        request.query = query
        response = ga_service.search(request=request)
        customer = list(response)[0].customer
        print('Credentials valid!')
      except Exception as e:
        logging.error("Connecting to GoogleAds failed %s", e)
        print('Credentials invalid!')
    else:
      print('Missing Credentials!')

button.on_click(on_button_clicked)
display(button, output)
creds = Credentials.from_authorized_user_info(config)


In [ ]:
# @title Load classes and instantiate clients.


class BulkTemplateType(enum.Enum):
  KEYWORD = [
      'Action',
      'Customer ID',
      'Campaign',
      'Ad group',
      'Keyword',
      'Original Keyword',
      'Match Type',
      'Keyword status',
      'Label',
  ]
  CAMPAIGN = [
      'Action',
      'Campaign status',
      'Customer ID',
      'Campaign',
      'Campaign type',
      'Bid strategy type',
      'Label',
  ]
  ADGROUP = ['Action', 'Customer ID', 'Campaign', 'Ad group', 'Status', 'Label']
  AD = [
      'Action',
      'Ad status',
      'Campaign',
      'Ad Group',
      'Headline 1',
      'Original Headline 1',
      'Headline 2',
      'Original Headline 2',
      'Headline 3',
      'Original Headline 3',
      'Headline 4',
      'Original Headline 4',
      'Headline 5',
      'Original Headline 5',
      'Headline 6',
      'Original Headline 6',
      'Headline 7',
      'Original Headline 7',
      'Headline 8',
      'Original Headline 8',
      'Headline 9',
      'Original Headline 9',
      'Headline 10',
      'Original Headline 10',
      'Headline 11',
      'Original Headline 11',
      'Headline 12',
      'Original Headline 12',
      'Headline 13',
      'Original Headline 13',
      'Headline 14',
      'Original Headline 14',
      'Headline 15',
      'Original Headline 15',
      'Description 1',
      'Original Description 1',
      'Description 2',
      'Original Description 2',
      'Description 3',
      'Original Description 3',
      'Description 4',
      'Original Description 4',
      'Final URL',
      'Label',
  ]


# Maximum number of segments accepted by the translate API
_TRANSLATE_BATCH_SIZE = 128


class Builder(object):

  def __init__(self, client, customer_id):
    self._service = client.get_service('GoogleAdsService')
    self._client = client
    self._customer_id = customer_id

  def _get_rows(self, query):
    search_request = self._client.get_type('SearchGoogleAdsStreamRequest')
    search_request.customer_id = self._customer_id
    search_request.query = query
    response = self._service.search_stream(request=search_request)
    return response


class GoogleAdsMccWorker(Builder):

  def __init__(self, client):
    super().__init__(client, client.login_customer_id)
    self._client = client

  def get_accounts(self, with_names=False):
    """Used to get all client accounts using API"""
    accounts = []
    query = """
    SELECT
      customer_client.descriptive_name,
      customer_client.id
    FROM
      customer_client
    WHERE
      customer_client.manager = False
    AND customer_client.status = 'ENABLED'
  """

    rows = self._get_rows(query)
    for batch in rows:
      for row in batch.results:
        # row = row._pb
        account = str(row.customer_client.id)
        if with_names:
          account += ' - ' + str(row.customer_client.descriptive_name)
        accounts.append(account)

    return accounts


class GoogleAdsAccWorker(Builder):

  def __init__(self, client, customer_id):
    super().__init__(client, customer_id)
    self._client = client

  def get_campaigns(self, with_names=False):
    """Used to get all client accounts using API"""
    campaigns_for_account = []
    query = """
      SELECT
        campaign.name,
        campaign.id
      FROM
        campaign
      WHERE
        campaign.status = 'ENABLED'
    """

    rows = self._get_rows(query)
    for batch in rows:
      for row in batch.results:
        # row = row._pb
        campaign = str(row.campaign.id)
        if with_names:
          campaign += ' - ' + str(row.campaign.name)
        campaigns_for_account.append(campaign)
    return campaigns_for_account

  def get_keywords_for_campaigns(
      self,
      campaign_ids: Optional[list[Union[int, str]]] = [],
      kw_statuses: Optional[list[str]] = ['ENABLED'],
      campaign_statuses: Optional[list[str]] = ['ENABLED'],
      ad_group_statuses: Optional[list[str]] = ['ENABLED'],
  ):
    """Used to get keywords including customer, campaign and ad group info."""
    query = f"""
        SELECT
            customer.id,
            campaign.name,
            campaign.advertising_channel_type,
            campaign.bidding_strategy_type,
            ad_group.name,
            ad_group_criterion.keyword.text,
            ad_group_criterion.keyword.match_type
        FROM keyword_view
        WHERE
            campaign.status in (
                {", ".join([f"'{elem}'" for elem in campaign_statuses])})
            AND ad_group.status in (
                {", ".join([f"'{elem}'" for elem in ad_group_statuses])})
            AND ad_group_criterion.status in (
                {", ".join([f"'{elem}'" for elem in kw_statuses])})
        """
    if campaign_ids:
      query += f"""AND campaign.id in ({", ".join([f"'{elem}'" for elem in campaign_ids])})"""
    rows = self._get_rows(query)
    keywords_data = []
    for batch in rows:
      for row in batch.results:
        # row = row._pb
        keywords_data.append([
            str(row.customer.id),
            str(row.campaign.name),
            str(row.campaign.advertising_channel_type).split('.')[1],
            str(row.campaign.bidding_strategy_type).split('.')[1],
            str(row.ad_group.name),
            str(row.ad_group_criterion.keyword.text),
            str(row.ad_group_criterion.keyword.match_type).split('.')[1],
        ])
    return keywords_data

  def _get_text_from_ad_assets_with_index(
      self, assets: list[str], index: int
  ) -> str:
    """Used to get text from ad assets."""
    text_list = []
    for asset in assets:
      if asset.text:
        text_list.append(asset.text)
    try:
      return text_list[index]
    except IndexError:
      return ''

  def get_ads_for_campaigns(
      self,
      campaign_ids: Optional[list[Union[int, str]]] = [],
      campaign_statuses: Optional[list[str]] = ['ENABLED'],
      ad_group_statuses: Optional[list[str]] = ['ENABLED'],
      ad_statuses: Optional[list[str]] = ['ENABLED'],
  ):
    """Used to get keywords including customer, campaign and ad group info."""
    query = f"""
        SELECT
            customer.id,
            campaign.name,
            ad_group.name,
            ad_group_ad.ad.responsive_search_ad.headlines,
            ad_group_ad.ad.responsive_search_ad.descriptions,
            ad_group_ad.ad.final_urls
        FROM ad_group_ad
        WHERE
            ad_group_ad.ad.type = RESPONSIVE_SEARCH_AD
            AND campaign.status in (
                {", ".join([f"'{elem}'" for elem in campaign_statuses])})
            AND ad_group.status in (
                {", ".join([f"'{elem}'" for elem in ad_group_statuses])})
            AND ad_group_ad.status in (
                {", ".join([f"'{elem}'" for elem in ad_statuses])})
        """
    if campaign_ids:
      query += f"""AND campaign.id in ({", ".join([f"'{elem}'" for elem in campaign_ids])})"""
    rows = self._get_rows(query)
    ad_data = []
    for batch in rows:
      for row in batch.results:
        # row = row._pb
        ad_data.append([
            str(row.customer.id),
            str(row.campaign.name),
            str(row.ad_group.name),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 0
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 1
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 2
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 3
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 4
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 5
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 6
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 7
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 8
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 9
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 10
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 11
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 12
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 13
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.headlines, 14
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.descriptions, 0
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.descriptions, 1
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.descriptions, 2
            ),
            self._get_text_from_ad_assets_with_index(
                row.ad_group_ad.ad.responsive_search_ad.descriptions, 3
            ),
            str(row.ad_group_ad.ad.final_urls[0]),
        ])
    return ad_data

  def get_active_keywords(self):
    """Used to get keywords for campaigns for deduplication."""
    query = """
      SELECT
          ad_group_criterion.keyword.text
      FROM ad_group_criterion
      WHERE
          campaign.status = 'ENABLED'
          AND ad_group.status = 'ENABLED'
          AND ad_group_criterion.type = 'KEYWORD'
    """
    keywords = []
    rows = self._get_rows(query)
    print(rows)
    for batch in rows:
      for row in batch.results:
        keywords.append(row.ad_group_criterion.keyword.text)
    keywords = list(set(keywords))
    return keywords


class SheetsWorker:
  """Class to handle sheets work."""

  def __init__(self) -> None:
    """Initializes the SheetsWorker class."""
    self._client = gspread.authorize(creds)

  def _format_for_spreadsheet(
      self, df: pd.DataFrame, template_type: BulkTemplateType
  ) -> list[list[Any]]:
    """Formats the dataframe to the required format."""

    if template_type == BulkTemplateType.KEYWORD:
      values = [BulkTemplateType.KEYWORD.value]
      for _, row in df.iterrows():
        values.append([
            'Add',
            'Enter Customer ID here',
            row.campaign_name,
            row.adgroup_name,
            row.keyword_text,
            row.original_keyword_text,
            row.match_type,
            'PAUSED',
            'keyword factory',
        ])
      return values

    if template_type == BulkTemplateType.ADGROUP:
      values = [BulkTemplateType.ADGROUP.value]
      for _, row in df.iterrows():
        values.append([
            'Add',
            'Enter Customer ID here',
            row.campaign_name,
            row.adgroup_name,
            'PAUSED',
            'keyword factory',
        ])
      return list(values for values, _ in itertools.groupby(values))

    if template_type == BulkTemplateType.CAMPAIGN:
      values = [BulkTemplateType.CAMPAIGN.value]
      for _, row in df.iterrows():
        values.append([
            'Add',
            'PAUSED',
            'Enter Customer ID here',
            row.campaign_name,
            row.advertising_channel_type,
            row.bidding_strategy_type,
            'keyword factory',
        ])
      return list(values for values, _ in itertools.groupby(values))

    if template_type == BulkTemplateType.AD:
      values = [BulkTemplateType.AD.value]
      for _, row in df.iterrows():
        values.append([
            'Add',
            'PAUSED',
            row.campaign_name,
            row.adgroup_name,
            row.headline_1,
            row.original_headline_1,
            row.headline_2,
            row.original_headline_2,
            row.headline_3,
            row.original_headline_3,
            row.headline_4,
            row.original_headline_4,
            row.headline_5,
            row.original_headline_5,
            row.headline_6,
            row.original_headline_6,
            row.headline_7,
            row.original_headline_7,
            row.headline_8,
            row.original_headline_8,
            row.headline_9,
            row.original_headline_9,
            row.headline_10,
            row.original_headline_10,
            row.headline_11,
            row.original_headline_11,
            row.headline_12,
            row.original_headline_12,
            row.headline_13,
            row.original_headline_13,
            row.headline_14,
            row.original_headline_14,
            row.headline_15,
            row.original_headline_15,
            row.description_1,
            row.original_description_1,
            row.description_2,
            row.original_description_2,
            row.description_3,
            row.original_description_3,
            row.description_4,
            row.original_description_4,
            row.final_url,
            'keyword factory',
        ])
      return values

  def write_to_spreadsheet(
      self,
      translated_kw_df: pd.DataFrame,
      translated_ads_df: pd.DataFrame,
      spreadsheet_id: Optional[str] = '',
  ):
    if spreadsheet_id:
      spreadsheet = self._client.open_by_key(spreadsheet_id)
    else:
      spreadsheet = self._client.create('Keyword Factory Upload')
      print(f'Created spreadsheet with id {spreadsheet.id}.')

    timestamp = time.strftime('%Y%m%d-%H%M%S')

    for template in BulkTemplateType:
      output_worksheet_name = f'{template.name}_upload_{timestamp}'
      if template != BulkTemplateType.AD:
        values = self._format_for_spreadsheet(translated_kw_df, template)
      else:
        values = self._format_for_spreadsheet(translated_ads_df, template)

      if values:
        output_worksheet = spreadsheet.add_worksheet(
            title=output_worksheet_name,
            rows=f'{len(values) + 1}',
            cols=f'{len(template.value)}',
        )
        output_worksheet.update(values)
        print(f'Added {len(values)} rows to sheet: {output_worksheet}.')
      else:
        print(f'No rows to add to sheet: {output_worksheet}.')

    worksheet_to_delete = spreadsheet.worksheet('Sheet1')
    spreadsheet.del_worksheet(worksheet_to_delete)

    return spreadsheet.id


class TranslateWorker:
  """Class with utils using Google Cloud Translate API."""

  def __init__(self, client, cloud_project_id: str) -> None:
    self._client = client
    self._parent = f'projects/{cloud_project_id}'

  def translate_list(
      self, batch_text: list[str], target_lang: str, source_lang: str = 'en'
  ) -> list[str]:
    """Returns the translated text."""

    empty_string_indexes = [i for i, x in enumerate(batch_text) if x == '']

    non_empty_strings = []
    for s in batch_text:
      if s:
        non_empty_strings.append(s)

    num_batches = math.ceil(len(non_empty_strings) / _TRANSLATE_BATCH_SIZE)
    progress_bar = alive_it(
        range(0, num_batches),
        stats=False,
        title='Translating Product Batch Text',
    )

    responses = []
    for batch_idx in progress_bar:
      batch_start = batch_idx * _TRANSLATE_BATCH_SIZE
      batch_end = batch_start + _TRANSLATE_BATCH_SIZE
      batch = non_empty_strings[batch_start:batch_end]

      response = self._client.translate_text(
          parent=self._parent,
          contents=batch,
          target_language_code=target_lang,
          source_language_code=source_lang,
          mime_type='text/plain',
          )
      logging.info(response)
      responses.extend([t.translated_text for t in response.translations])

    for i in empty_string_indexes:
      responses.insert(i, '')
    return responses

  def translate_keyword_report(
      self,
      keyword_report: pd.DataFrame,
      target_lang: str,
      source_lang: str = 'en',
      target_customer_id: Optional[str] = None,
  ):
    df = keyword_report.copy()

    # Rename campaigns and adgroups according to the target language.
    df['campaign_name'] = keyword_report['campaign_name'] + f' ({target_lang})'
    df['adgroup_name'] = keyword_report['adgroup_name'] + f' ({target_lang})'
    # Translate keywords.
    try:
      df['original_keyword_text'] = df['keyword_text']
      translated_keywords = self.translate_list(
          batch_text=list(df['keyword_text']),
          source_lang=source_lang,
          target_lang=target_lang,
      )
      df['keyword_text'] = translated_keywords
    except Exception as e:
      logging.exception(e)
      raise e

    # Set customer_id to target customer id if provided.
    df['customer_id'] = (
        target_customer_id if target_customer_id else df['customer_id']
    )
    return df

  def translate_ad_report(
      self,
      ad_report: pd.DataFrame,
      target_lang: str,
      source_lang: str = 'en',
      target_customer_id: Optional[str] = None,
  ):
    df = ad_report.copy()
    cols = [
        'headline_1',
        'headline_2',
        'headline_3',
        'headline_4',
        'headline_5',
        'headline_6',
        'headline_7',
        'headline_8',
        'headline_9',
        'headline_10',
        'headline_11',
        'headline_12',
        'headline_13',
        'headline_14',
        'headline_15',
        'description_1',
        'description_2',
        'description_3',
        'description_4',
    ]
    # Rename campaigns and adgroups according to the target language.
    df['campaign_name'] = ad_report['campaign_name'] + f' ({target_lang})'
    df['adgroup_name'] = ad_report['adgroup_name'] + f' ({target_lang})'
    # Translate keywords.
    try:
      for col in cols:
        original_col = f'original_{col}'
        df[original_col] = df[col]
        if not all(s == '' for s in list(df[col])):
          translated_text = self.translate_list(
              batch_text=list(df[col]),
              source_lang=source_lang,
              target_lang=target_lang,
          )
          df[col] = translated_text
    except Exception as e:
      logging.exception(e)
      raise e

    # Set customer_id to target customer id if provided.
    df['customer_id'] = (
        target_customer_id if target_customer_id else df['customer_id']
    )
    return df


# Some helper functions.
def intersection(lst1, lst2):
  lst3 = [value for value in lst1 if value in lst2]
  return lst3


def filter_campaigns_map(campaigns_map, campaigns):
  map_to_process = {}
  for acc in campaigns_map:
    intersect = intersection(campaigns_map[acc], campaigns)
    if intersect:
      map_to_process[acc] = [x.split(' - ')[0] for x in intersect]
  return map_to_process


def get_keywords_data(
    client: GoogleAdsClient, map_to_process: dict[str, list[str]]
):
  keywords_data = []
  with futures.ThreadPoolExecutor() as executor:
    results = executor.map(
        lambda account: GoogleAdsAccWorker(
            client, account
        ).get_keywords_for_campaigns(campaign_ids=map_to_process[account]),
        map_to_process,
    )
    for result in results:
      if isinstance(result, list):
        keywords_data += result
  cols = [
      'customer_id',
      'campaign_name',
      'advertising_channel_type',
      'bidding_strategy_type',
      'adgroup_name',
      'keyword_text',
      'match_type',
  ]

  return pd.DataFrame(data=keywords_data, columns=cols)


def get_ads_data(client: GoogleAdsClient, map_to_process: dict[str, list[str]]):
  ads_data = []
  with futures.ThreadPoolExecutor() as executor:
    results = executor.map(
        lambda account: GoogleAdsAccWorker(
            client, account
        ).get_ads_for_campaigns(campaign_ids=map_to_process[account]),
        map_to_process,
    )
    for result in results:
      if isinstance(result, list):
        ads_data += result
  cols = [
      'customer_id',
      'campaign_name',
      'adgroup_name',
      'headline_1',
      'headline_2',
      'headline_3',
      'headline_4',
      'headline_5',
      'headline_6',
      'headline_7',
      'headline_8',
      'headline_9',
      'headline_10',
      'headline_11',
      'headline_12',
      'headline_13',
      'headline_14',
      'headline_15',
      'description_1',
      'description_2',
      'description_3',
      'description_4',
      'final_url',
  ]

  return pd.DataFrame(data=ads_data, columns=cols)


def depulicate_translated_keywords_df(
    df: pd.DataFrame,
    client: GoogleAdsClient,
    target_account: Optional[int] = None,
) -> pd.DataFrame:
  if target_account:
    keywords = GoogleAdsAccWorker(client, target_account).get_active_keywords()
    return df[~df['keyword_text'].isin(keywords)]
  else:
    return df


def extract_spreadsheet_id(url):
  """Given a Google spreadsheet URL, extracts the spreadsheet ID and returns it.

  Args:
      url (str): The URL of the Google spreadsheet.

  Returns:
      str: The ID of the Google spreadsheet.
  """
  # Extract the spreadsheet ID from the URL using a regular expression
  match = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
  if match:
    return match.group(1)
  else:
    raise ValueError('Invalid Google spreadsheet URL')


class Language(enum.Enum):
  AF = 'Afrikaans'
  AK = 'Akan'
  SQ = 'Albanian'
  AM = 'Amharic'
  AR = 'Arabic'
  HY = 'Armenian'
  AS = 'Assamese'
  AY = 'Aymara'
  AZ = 'Azerbaijani'
  BM = 'Bambara'
  EU = 'Basque'
  BE = 'Belarusian'
  BN = 'Bengali'
  BHO = 'Bhojpuri'
  BS = 'Bosnian'
  BG = 'Bulgarian'
  CA = 'Catalan'
  CEB = 'Cebuano'
  NY = 'Chichewa'
  ZH = 'Chinese (Simplified)'
  ZH_TW = 'Chinese (Traditional)'
  CO = 'Corsican'
  HR = 'Croatian'
  CS = 'Czech'
  DA = 'Danish'
  DV = 'Divehi'
  DOI = 'Dogri'
  NL = 'Dutch'
  EN = 'English'
  EO = 'Esperanto'
  ET = 'Estonian'
  EE = 'Ewe'
  TL = 'Filipino'
  FI = 'Finnish'
  FR = 'French'
  FY = 'Frisian'
  GL = 'Galician'
  LG = 'Ganda'
  KA = 'Georgian'
  DE = 'German'
  GOM = 'Goan Konkani'
  EL = 'Greek'
  GN = 'Guarani'
  GU = 'Gujarati'
  HT = 'Haitian Creole'
  HA = 'Hausa'
  HAW = 'Hawaiian'
  IW = 'Hebrew'
  HI = 'Hindi'
  HMN = 'Hmong'
  HU = 'Hungarian'
  IS = 'Icelandic'
  IG = 'Igbo'
  ILO = 'Iloko'
  ID = 'Indonesian'
  GA = 'Irish'
  IT = 'Italian'
  JA = 'Japanese'
  JW = 'Javanese'
  KN = 'Kannada'
  KK = 'Kazakh'
  KM = 'Khmer'
  RW = 'Kinyarwanda'
  KO = 'Korean'
  KRI = 'Krio'
  KU = 'Kurdish (Kurmanji)'
  CKB = 'Kurdish (Sorani)'
  KY = 'Kyrgyz'
  LO = 'Lao'
  LA = 'Latin'
  LV = 'Latvian'
  LN = 'Lingala'
  LT = 'Lithuanian'
  LB = 'Luxembourgish'
  MK = 'Macedonian'
  MAI = 'Maithili'
  MG = 'Malagasy'
  MS = 'Malay'
  ML = 'Malayalam'
  MT = 'Maltese'
  MNI_MTEI = 'Manipuri (Meitei Mayek)'
  MI = 'Maori'
  MR = 'Marathi'
  LUS = 'Mizo'
  MN = 'Mongolian'
  MY = 'Myanmar (Burmese)'
  NE = 'Nepali'
  NSO = 'Northern Sotho'
  NO = 'Norwegian'
  OR = 'Odia (Oriya)'
  OM = 'Oromo'
  PS = 'Pashto'
  FA = 'Persian'
  PL = 'Polish'
  PT = 'Portuguese'
  PA = 'Punjabi'
  QU = 'Quechua'
  RO = 'Romanian'
  RU = 'Russian'
  SM = 'Samoan'
  SA = 'Sanskrit'
  GD = 'Scots Gaelic'
  SR = 'Serbian'
  ST = 'Sesotho'
  SN = 'Shona'
  SD = 'Sindhi'
  SI = 'Sinhala'
  SK = 'Slovak'
  SL = 'Slovenian'
  SO = 'Somali'
  ES = 'Spanish'
  SU = 'Sundanese'
  SW = 'Swahili'
  SV = 'Swedish'
  TG = 'Tajik'
  TA = 'Tamil'
  TT = 'Tatar'
  TE = 'Telugu'
  TH = 'Thai'
  TI = 'Tigrinya'
  TS = 'Tsonga'
  TR = 'Turkish'
  TK = 'Turkmen'
  UK = 'Ukrainian'
  UR = 'Urdu'
  UG = 'Uyghur'
  UZ = 'Uzbek'
  VI = 'Vietnamese'
  CY = 'Welsh'
  XH = 'Xhosa'
  YI = 'Yiddish'
  YO = 'Yoruba'
  ZU = 'Zulu'
  HE = 'Hebrew'
  JV = 'Javanese'
  ZH_CN = 'Chinese (Simplified)'


# Instantiating all clients.
translate_client = translate_v3.TranslationServiceClient(credentials=creds)
print('Successfully instantiated Cloud Translation API Client')
ads_client = GoogleAdsClient.load_from_dict(config)
print('Successfully instantiated Google Ads API Client')
sheets_client = SheetsWorker()
print('Successfully instantiated GSpread Client')

# Run

In [ ]:
#@title Select Account(s)
#@markdown Select the accounts that contain the campaigns with keywords that you want to translate.
#@markdown
#@markdown Hold the **Shift** button so select multiple accounts.
mcc_worker = GoogleAdsMccWorker(ads_client)
accounts = mcc_worker.get_accounts(True)
accounts_picker = widgets.SelectMultiple(
    options=accounts,
    description='Accounts',
    disabled=False,
    layout=widgets.Layout(width='25%', height='300px')
)

accounts_picker

In [ ]:
#@title Select Campaign(s)
#@markdown Select the campaigns with keywords that you want to translate.
#@markdown All enabled AdGroups will be pulled in automatically.
#@markdown
#@markdown To select multiple campaigns hold **Shift** to select multiple campaigns.
selected_accounts = [x.split(' - ')[0] for x in accounts_picker.value]

campaigns_map = dict()

for acc in selected_accounts:
    acc_worker = GoogleAdsAccWorker(ads_client, acc)
    campaigns_map[acc] = acc_worker.get_campaigns(True)

campaigns = []
for acc in campaigns_map:
  campaigns.extend(campaigns_map[acc])

campaigns_picker = widgets.SelectMultiple(
    options=campaigns,
    description='Campaigns',
    disabled=False,
    layout=widgets.Layout(width='50%', height='300px')
)

campaigns_picker

In [ ]:
#@title Select Languages(s)
#@markdown Specify the source and target language for translation.
source_language = 'English' #@param ['Afrikaans', 'Akan', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Aymara', 'Azerbaijani', 'Bambara', 'Basque', 'Belarusian', 'Bengali', 'Bhojpuri', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chichewa', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Divehi', 'Dogri', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Ewe', 'Filipino', 'Finnish', 'French', 'Frisian', 'Galician', 'Ganda', 'Georgian', 'German', 'Goan Konkani', 'Greek', 'Guarani', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Iloko', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Kinyarwanda', 'Korean', 'Krio', 'Kurdish (Kurmanji)', 'Kurdish (Sorani)', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Maithili', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Manipuri (Meitei Mayek)', 'Maori', 'Marathi', 'Mizo', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Northern Sotho', 'Norwegian', 'Odia (Oriya)', 'Oromo', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Quechua', 'Romanian', 'Russian', 'Samoan', 'Sanskrit', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tigrinya', 'Tsonga', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uyghur', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
target_language = 'Spanish' #@param ['Afrikaans', 'Akan', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Aymara', 'Azerbaijani', 'Bambara', 'Basque', 'Belarusian', 'Bengali', 'Bhojpuri', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chichewa', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Divehi', 'Dogri', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Ewe', 'Filipino', 'Finnish', 'French', 'Frisian', 'Galician', 'Ganda', 'Georgian', 'German', 'Goan Konkani', 'Greek', 'Guarani', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Iloko', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Kinyarwanda', 'Korean', 'Krio', 'Kurdish (Kurmanji)', 'Kurdish (Sorani)', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Maithili', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Manipuri (Meitei Mayek)', 'Maori', 'Marathi', 'Mizo', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Northern Sotho', 'Norwegian', 'Odia (Oriya)', 'Oromo', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Quechua', 'Romanian', 'Russian', 'Samoan', 'Sanskrit', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tigrinya', 'Tsonga', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uyghur', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
#@markdown Optional: Add a Target Spreadsheet ID or URL
spreadsheet_url = '' #@param  {type:"string"}
spreadsheet_id = ''
if spreadsheet_url:
  spreadsheet_id = extract_spreadsheet_id(spreadsheet_url)

#@markdown Optional: Add a target account used for deduplication.
target_customer_id = "" #@param {type:"string"}

In [ ]:
#@title Run Translator
source_lang = Language(source_language).name.replace('_', '-').lower()
target_lang = Language(target_language).name.replace('_', '-').lower()

map_to_process = filter_campaigns_map(campaigns_map, campaigns_picker.value)

keywords_dataframe = get_keywords_data(ads_client, map_to_process)

ads_dataframe = get_ads_data(ads_client, map_to_process)

translated_kw_dataframe = TranslateWorker(
    translate_client, cloud_project_id).translate_keyword_report(
        keyword_report=keywords_dataframe,
        source_lang=source_lang,
        target_lang=target_lang,
        target_customer_id=target_customer_id)

translated_kw_dataframe = depulicate_translated_keywords_df(
    df=translated_kw_dataframe,
    client=ads_client,
    target_account=target_customer_id)

translated_ad_dataframe = TranslateWorker(
    translate_client, cloud_project_id).translate_ad_report(
        ads_dataframe, source_lang=source_lang, target_lang=target_lang)

spreadsheet_id = SheetsWorker().write_to_spreadsheet(translated_kw_dataframe, translated_ad_dataframe)
print(f'Go the spreadsheet by clicking this link: https://docs.google.com/spreadsheets/d/{spreadsheet_id}')